# 2.1 Explicit Intent

W tym przykładzie przyjrzymy się mechanizmowi intentów, rozpoczniemy od intentów jawnych. Stworzymy aplikację z dwiema aktywnościami i umożliwimy otworzenie drugiej aktywności z poziomu pierwszej (głównej). 

Przedźmy do aktywności głównej i dodajmy przycisk do layoutu.

### Layout i utworzenie drugiej aktywności

```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".MainActivity">

    <Button
        android:id="@+id/button"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Button"
        android:layout_margin="16dp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        android:onClick="startSecondActivity"/>
</androidx.constraintlayout.widget.ConstraintLayout>
```

Po naciśnięciu tego przycisku możliwe będzie przejście do drugiej aktywności.

Następnie dodajmy drugą aktywność do projektu. W oknie **Project** wybieram z menu kontekstowego **New -> Activity -> Empty Activity**. Zaznaczam **Generate layout file** i odznaczam **Launcher Activity**. Przejdźmy do pliku `AndroidManifest.xml`

Mamy opisane dwie aktywności. Chcemy teraz wskazać rodzica dla **SecondActivity** . Do tagu `<activity />` dodaję opcję `android:parentActivityName=".MainActivity"` oraz `android:label="Second Activity"`.

```xml
<activity
    android:name=".SecondActivity"
    android:parentActivityName=".MainActivity"
    android:label="Second Activity"
    android:exported="false" />
```

Czyli wskazaliśmy ze aktywnością nadrzędną jest **MainActivity**. Zwróćmy również uwagę na kropkę przed nazwą aktywności głównej - pominięta została nazwa pakietu (`pl.edu.uwr.pum.explicitintentjava.MainActivity`). `label` posłuży jako wyświetlana nazwa aktywności.

Teraz utworzymy prosty layout dla drugiej aktywności - przechodzimy do pliku `activity_second.xml` i dodajemy pole `TextView`.

```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".SecondActivity">

    <TextView
        android:id="@+id/textView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Second Activity"
        android:textSize="40sp"
        android:textStyle="bold"
        android:textColor="@color/black"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />
</androidx.constraintlayout.widget.ConstraintLayout>
```

### Utworzenie `Intent` i przejście do drugiej aktywności

Dodajmy teraz obsługę przycisku w naszej głównej aktywności, przyjdźmy do pliku `MainActivity.java`. Po wciśnięciu przycisku chcemy przejść do drugiej aktywności.

In [ ]:
public void startSecondActivity(View view) {
        
}

W pierwszym kroku musimy utworzyć **intent**.

In [ ]:
Intent intent = new Intent(this, SecondActivity.class);

Konstruktor klasy `Intent` przyjmuje dwa argumenty. Pierwszym jest **context**, ponieważ aktywność jest swoim własnym kontekstem używamy słowa kluczowego `this`. Drugim argumentem jest klasa aktywności którą chcemy włączyć - tutaj będzie to `SecondActivity.class`.

Aby włączyć aktywność wykonujemy metodę `startActivity`, jako argumentb podajemy poprzednio utworzony `Intent`.

In [ ]:
startActivity(intent);

Na tym etapie możemy zbudować nasz projekt i przetestować funkcjonalność przycisku. Po przejściu do drugiej aktywności zwróćmy uwagę na przycisk powrotu znajdujący się na belce po lewej stronie.

<img src="https://fv9-4.failiem.lv/thumb_show.php?i=vset6gbuc&view" width="200"/>

Nawigację do aktywności nadrzędnej mamy dodaną automatyvcznie dzięki dodaniu do manifestu linii `android:parentActivityName=".MainActivity"`

## **Przekazanie danych do drugiej aktywności**

Kolejnym krokiem będzie przekazanie danych i ich odbiór w drugiej aktywności - tutaj przekażemy tekst z pola `EditText`. Do layoutu głównej aktywności dodajmy odpowiednie pole.

```xml
    <EditText
        android:id="@+id/editTextMain"
        android:layout_width="0dp"
        android:layout_height="wrap_content"
        android:layout_margin="16dp"
        android:hint="message"
        android:textStyle="bold"
        android:textSize="24sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toStartOf="@+id/button"
        app:layout_constraintStart_toStartOf="parent" />
```

Pierwszą rzeczą jakiej będziemy potrzebować jest to **klucz** naszej wiadomości. Przejdźmy do klasy głównej aktyności `MainActivity.java` i dodajmy odpowiednie pole. Standardowo klucze przechowuje się w finalnych polach statycznych klasy wysyłającej.

In [ ]:
public static final String EXTRA_MESSAGE = "pl.edu.uwr.pum.explicitintentjava.MESSAGE";

Standardowo nazwę stałej przechowującej klucz piszemy wielkimi literami, sam `String` zawierający klucz składa się z pakietu w której znajduje się aktywność wysyłająca i nazwy.

Dalej utwórzmy pole reprezentujące `EditText` i połączmy z elementem layoutu o odpowiednim `id`.

In [ ]:
private EditText editText;

@Override
protected void onCreate(Bundle savedInstanceState) {
    super.onCreate(savedInstanceState);
    setContentView(R.layout.activity_main);
        
    editText = findViewById(R.id.editTextMain);
}

Ostatnim krokiem będzie spakowanie danych i przesłanie ich do drugiej aktywności. Przechodzimy do metody `startSecondActivity` (czyli metody `onClick` naszego przycisku) i uzupełniamy implementację.

W pierwszym kroku wyciągamy tekst z pola `EditText`

In [ ]:
String message = editText.getText().toString();

Następnie dodajemy do intentu nasze dane - czyli `String` wyciągnięty z `EditText`. W tym celu na naszym obiekcie `Intent` wywołujemy metodę `putExtra`, która dodaje dane do obiektu `Intent` i przekazuje je do aktywności odbierającej.

In [ ]:
intent.putExtra(EXTRA_MESSAGE, message);

Metoda przyjmuje dwa argumenty. Pierwszym jest nasz unikalny klucz, który mamy zapisany w stałej `EXTRA_MESSAGE`, drugim jest `String` który chcemy przekazać.

In [ ]:
public void startSecondActivity(View view) {
    String message = editText.getText().toString();
    Intent intent = new Intent(this, SecondActivity.class);
    intent.putExtra(EXTRA_MESSAGE, message);
    startActivity(intent);
}

## **Odebranie danych z poziomu drugiej aktywności**

Kolejnym krokiem będzie odebranie danych z poziomu drugiej aktywności - odebrany `String` przekażemy do pola `TextView`. Przechodzimy do pliku `SecondActivity.java` i w metodzie `onCreate` tworzymy `Intent` i wywołujemy metodę `getIntent` - dzięki tej metodzie przesłany `Intent` z aktywności głównej zostanie odebrany.

In [ ]:
Intent intent = getIntent();

Następnie odbieramy przesłane dane za pomocą metody `getStringExtra` wywołanej na odebranej instancji klasy `Intent`.

In [ ]:
String message = intent.getStringExtra(MainActivity.EXTRA_MESSAGE);

Metoda jako argument przyjmuje unikalny klucz, który mamy zapisany w stałej statycznej klasy `MainActivity`. W ostatnim kroku chcemy odebrany `String` przekazać do pola `TextView`.

In [ ]:
package pl.edu.uwr.pum.explicitintentjava;

import androidx.appcompat.app.AppCompatActivity;

import android.content.Intent;
import android.os.Bundle;
import android.widget.TextView;

public class SecondActivity extends AppCompatActivity {

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_second);

        Intent intent = getIntent();
        String message = intent.getStringExtra(MainActivity.EXTRA_MESSAGE);
        TextView textView = findViewById(R.id.textView);
        textView.setText(message);
    }
}

Na tym etapie możemy przetestować funkcjonalność aplikacji.

## **Przekazanie odpowiedzi do aktywności głównej**

Kontynuujemy aplikację, kolejnym krokiem będzie zarządanie odpowiedzi z drugiej zktywności. Rozpocznijmy od zmodyfikowania layoutu drugiej aktywności - dodajmy pole `EditText` w którym wpiszemy naszą odpowiedź oraz `Button` w celu powrotu i zainicjowania powrotnego przesłania danych.

```xml
    <EditText
        android:id="@+id/editTextSecond"
        android:layout_width="0dp"
        android:layout_height="wrap_content"
        android:layout_margin="16dp"
        android:hint="Back message"
        android:textStyle="bold"
        android:textSize="24sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toStartOf="@+id/button"
        app:layout_constraintStart_toStartOf="parent" />

    <Button
        android:id="@+id/returnButton"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Return"
        android:layout_margin="16dp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        android:onClick="returnMessage"/>
```

Przejdźmy do klasy `SecondActivity.java` i dodajmy pole statyczne przechowujące unikalny klucz naszej odpowiedzi.

In [ ]:
public static final String EXTRA_REPLY = "pl.edu.uwr.pum.explicitintentjava.REPLY";

Następnie dodajmy pole `EditText` do klasy.

In [ ]:
private EditText editText;

@Override
protected void onCreate(Bundle savedInstanceState) {
    super.onCreate(savedInstanceState);
    setContentView(R.layout.activity_second);
        
    editText = findViewById(R.id.editTextSecond);
    ...
}

Następnie dodajmy metodę `onClick` przycisku **return**.

In [ ]:
public void returnMessage(View view) {
    String reply = editText.getText().toString();
    Intent intent = new Intent();
        
    intent.putExtra(EXTRA_REPLY, reply);
    setResult(RESULT_OK, intent);
    finish();
}

Podobnie jak przy wysyłaniu wiadomości z głównej aktywności tworzymy `String` i wrzucamy do niego tekst odczytany z pola `EditText`. Następnie tworzymy `Intent` i wykorzystujemy metodę `putExtra` do dołączenia danych. Wywołujemy metodę `setResult` która przyjjmuje dwa argumenty. Pierwwzym jest kod - tutaj wykorzystamy kod `RESULT_OK`, czyli nie przewidujemy żadnych problemów z odczytaniem, jako drugi argument podajemy nasz `Intent`.

W metodzie możemy podać różne kody, które są predefiniowane w klasie `Activity` - `RESULT_OK`, `RESULT_CANCELED` i `RESULT_FIRST_USER`. Pierwszy dotyczy przypadku w których wszystko poszło po naszej myśli i możemy przekazać dane powrotne. Dzięki `RESULT_CANCELED` możemy sprecyzować co nasza aktywność główna ma zrobić gdy nie uda się wysłać danych z drugiej aktywności. Trzeci przypadek istnieje aby uniknąć konfliktów - pozwala developerowi na wykorzystanie własnego (nie predefiniowanego) kodu.

Na koniec wywołujemy metodę `finish` która kończyb działanie `SecondActivity`.

Następnie do layoutu `MainActivity` dodajmy pole `TextView` w którym wyświetlimy odebraną wiadomość.

```xml
    <TextView
        android:id="@+id/textViewMain"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Main Activity"
        android:textSize="40sp"
        android:textStyle="bold"
        android:textColor="@color/black"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />
```

I stwórzmy odpowiednie pole w klasie `MainActivity.java`.

In [ ]:
private EditText editText;
private TextView textView;

@Override
protected void onCreate(Bundle savedInstanceState) {
    super.onCreate(savedInstanceState);
    setContentView(R.layout.activity_main);

    editText = findViewById(R.id.editTextMain);
    textView = findViewById(R.id.textViewMain);
}

Ostatnim krokiem będzie modyfikacja klasy `MainActivity.java` w celu wysłania żądania i odebrania danych powrotnych.

W pierwszym kroku w metodzie `onClick` chcemy poozbyć się wywołania `startActivity`. Teraz nie chcemy w prosty sposób włączać innej aktywności, tylko wysłać do niej żądanie przekazania danych.

In [ ]:
public void startSecondActivity(View view) {
    String message = editText.getText().toString();
    Intent intent = new Intent(this, SecondActivity.class);
    intent.putExtra(EXTRA_MESSAGE, message);
    //startActivity(intent); // <- usuwamy
}

Musimy napisać launcher naszej drugiej aktywności.

In [ ]:
ActivityResultLauncher<Intent> secondActivityResultLauncher = registerForActivityResult(
        new ActivityResultContracts.StartActivityForResult(),
        new ActivityResultCallback<ActivityResult>() {
            @Override
            public void onActivityResult(ActivityResult result) {
                    
            }
        }
);


Tworzymy nowy `ActivityResultLauncher`, będziemy przekazywać `Intent` więc podajemy w sygnaturze `<Intent>` i wywołujęmy `registerForActivityResult` który przyjmuje dwa argumenty. 

Rejestrujemy prośbę o rozpoczęcie innej aktywności. Powoduje to utworzenie wpisu w rejestrze powiązanym z drugą aktywnością, zarządzającym kodem żądania a także konwersjami do/z `Intent`. Jako pierwszy parametr podajemy `Contract`, nas interesuje metoda `StartActivityForResult`, poprzednio włączaliśmy drugą aktywność metodą `startActivity` - teraz żądamy odpowiedzi więc konieczne jest wykorzystanie innej meetody. Drugim argumentem jest `Callback` czyli tutaj jest to dostarczenie implementacji metody `onActivityResult` - tutaj określamy co robimy z danymi które dostajemy powrotnie z aktywności `SecondActivity`. Dla uproszczenia można to wywołanie zastąpić wyrażeniem **lambda**.

In [ ]:
ActivityResultLauncher<Intent> secondActivityResultLauncher = registerForActivityResult(
        new ActivityResultContracts.StartActivityForResult(),
        result -> {

        }
);

`result` jest otrzymanym z drugiej aktywności `Intent` zawierającym dane. Następnie dodajjmy implementację metody `onActivityResult`

W pierwszym kroku sprawdzam czy aktywność zwraca z poprawnym kodem

In [ ]:
if (result.getResultCode() == Activity.RESULT_OK)

Następnie sprawdzamy czy dane istnieją i wyciągamy dane z `result` i umieszczamy w polu `textView`.

In [ ]:
ActivityResultLauncher<Intent> secondActivityResultLauncher = registerForActivityResult(
    new ActivityResultContracts.StartActivityForResult(),
    result -> {
        if (result.getResultCode() == Activity.RESULT_OK && 
            result.getData() != null)
            textView.setText(result.getData()
                             .getStringExtra(SecondActivity.EXTRA_REPLY));
        }
);

Ostatnim krokiem jest wykorzystanie `secondActivityResultLauncher` w metodzie `onClick` przycisku wysyłającego żądanie

In [ ]:
public void startSecondActivity(View view) {
    String message = editText.getText().toString();
    Intent intent = new Intent(this, SecondActivity.class);
    intent.putExtra(EXTRA_MESSAGE, message);
    secondActivityResultLauncher.launch(intent);
}

Możemy przetestowac aplikację.